<a href="https://colab.research.google.com/github/AlexisL321/band_member_detection_detectron2/blob/main/coding_task_frame.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import drive
drive.mount('/content/drive/', force_remount = True)

Mounted at /content/drive/


In [4]:
import glob
from imutils.video import FileVideoStream
import time
import numpy as np
import os
import cv2
import yaml
import matplotlib.pyplot as plt

In [ ]:
#Extract frames from video
pathName = '/content/drive/My Drive/Drone Footage_Coding Task/Drone Footage/11 9 flights/'
videos = glob.glob(pathName + '*.MP4')
videos = videos[10:-1]
for video in videos:
  numFrame = 0
  fvs = FileVideoStream(video).start()
  maxFrame = 500
  time.sleep(1.0)
  start = time.time()
  videoName = video.split('/')[7].split('.')[0]
  outFolder = '/content/drive/My Drive/Frames/'+videoName
  os.makedirs(outFolder, exist_ok = True)
  while fvs.more():
    frame = fvs.read()
    #print(frame)
    outFile = os.path.join(outFolder, "%s_%i.jpg"%(videoName, numFrame))
    #print(outFile)
    if np.any(frame):
      cv2.imwrite(outFile, frame)
    else:
      break
    if numFrame % 1000 == 0:
      print(f"{numFrame} processed, {np.any(frame)}")

    numFrame += 1
    if numFrame > maxFrame:
      break
  
  totalTime = time.time() - start
  print("Total time: {:.2f}s", format(totalTime))


0 processed, True
Total time: {:.2f}s 145.8224196434021
0 processed, True
Total time: {:.2f}s 154.9542942047119
0 processed, True
Total time: {:.2f}s 153.94745755195618
0 processed, True


In [ ]:
#convert json files to coc json format
import json
from google.colab import files
src = list(files.upload().values())[0]
#open('COCOJson.py','wb').write(src)
#import labelbox_processing
#import COCOJson
#import json_functions


Saving inference.py to inference.py
Saving PrecisionRecallEvaluator.py to PrecisionRecallEvaluator.py
Saving LossEvalHook.py to LossEvalHook.py
Saving GroundtruthVisualizer.py to GroundtruthVisualizer.py
Saving LRScheduler.py to LRScheduler.py
Saving DetectionDatasetMapper.py to DetectionDatasetMapper.py
Saving DetectionTrainer.py to DetectionTrainer.py


In [ ]:
jsondir = '/content/drive/My Drive/Drone Footage_Coding Task/All Labels/'
jsonFiles = glob.glob(jsondir + "*.json")
os.makedirs('/content/drive/My Drive/coco/cocojson', exist_ok = True)
os.makedirs('/content/drive/My Drive/coco/images', exist_ok = True)
cocoFolder = '/content/drive/My Drive/coco/cocojson';

for jsonfile in jsonFiles:
  jsonfileName = os.path.basename(jsonfile)
  imageFolder = os.path.join('/content/drive/My Drive/coco/images', jsonfileName.split('_')[1])
  os.makedirs(imageFolder, exist_ok = True)
  cocojsonFile = os.path.join(cocoFolder, "coco_"+ jsonfileName)
  labelbox_processing.labelbox_to_coco(jsonfile, cocojsonFile, imageFolder, 
                                       description = jsonfileName.split('_')[1],
                                       date = "March,2023", overwrite = False, verbose = True,
                                       custom_class_reader = None)
  

1 band_member
2 object_e_g_backpacks_belongings
3 observer_non_band_member
saving at /content/drive/My Drive/coco/cocojson/coco_export-2023-02-28T19_22_17.380Z.json
4234 annotations from 11 images saved.
1 band_member
2 object_e_g_backpacks_belongings
3 observer_non_band_member
saving at /content/drive/My Drive/coco/cocojson/coco_export-2023-02-28T19_25_46.471Z.json
8670 annotations from 20 images saved.
1 band_member
2 object_e_g_backpacks_belongings
3 observer_non_band_member
saving at /content/drive/My Drive/coco/cocojson/coco_export-2023-02-28T19_26_37.868Z.json
5059 annotations from 15 images saved.
1 band_member
2 object_e_g_backpacks_belongings
3 observer_non_band_member
saving at /content/drive/My Drive/coco/cocojson/coco_export-2023-02-28T19_29_07.431Z.json
8068 annotations from 20 images saved.
1 band_member
2 object_e_g_backpacks_belongings
3 observer_non_band_member
saving at /content/drive/My Drive/coco/cocojson/coco_export-2023-03-02T01_21_56.571Z.json
8817 annotations fr

In [ ]:
# split the data
jsondir = '/content/drive/My Drive/coco/cocojson/'
jsonFiles = glob.glob(jsondir + "*.json")
jsonFileArray = [jsonfile for jsonfile in jsonFiles]
output_file = os.path.join(jsondir, "all_annotations1.json")
json_functions.combine_jsons(jsonFileArray, output_file)

fracValidation = 0.2
json_functions.create_train_val_split(output_file, fracValidation, 
                                      train_name= "train.json", 
                                      val_name = "val.json")

There are 555 annotated images in the JSON files.
555 images added to new .json
220385 annotations added to new .json
There are 555 annotated images.
444 training images with 176512 annotations.
111 validation images with 43873 annotations.


In [30]:
# install detectron2
!python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/facebookresearch/detectron2.git to /tmp/pip-req-build-j4115ls5
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/detectron2.git /tmp/pip-req-build-j4115ls5
  Resolved https://github.com/facebookresearch/detectron2.git to commit 072713649a9b0069c10aad1aaab819112e8f1e2e
  Preparing metadata (setup.py) ... done


In [33]:
!python -m pip install pyyaml==5.1
import sys, os, distutils.core
# Note: This is a faster way to install detectron2 in Colab, but it does not include all functionalities.
# See https://detectron2.readthedocs.io/tutorials/install.html for full installation instructions
!git clone 'https://github.com/facebookresearch/detectron2'
dist = distutils.core.run_setup("./detectron2/setup.py")
!python -m pip install {' '.join([f"'{x}'" for x in dist.install_requires])}
sys.path.insert(0, os.path.abspath('./detectron2'))

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Cloning into 'detectron2'...
remote: Enumerating objects: 14935, done.
remote: Counting objects: 100% (20/20), done.
remote: Compressing objects: 100% (17/17), done.
remote: Total 14935 (delta 7), reused 13 (delta 3), pack-reused 14915
Receiving objects: 100% (14935/14935), 6.05 MiB | 19.74 MiB/s, done.
Resolving deltas: 100% (10829/10829), done.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [60]:
import torch, detectron2
!nvcc --version
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
print("detectron2:", detectron2.__version__)

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0
torch:  1.13 ; cuda:  cu116


AttributeError: ignored

In [59]:
# build the model
import shutil
#shutil.move("/content/detectron2/detectron2/__init__.py", "/content/detectron2/")
from detectron2.utils.visualizer import Visualizer

#import shutil
#shutil.rmtree('/content/detectron2/detectron2')


In [2]:
imageFolder = '/content/drive/My Drive/coco/allImages'
trainFile = '/content/drive/My Drive/coco/cocojson/train.json'
validFile = '/content/drive/My Drive/coco/cocojson/val.json'
detectron2.data.datasets.register_coco_instances("train", {}, trainFile, imageFolder)
detectron2.data.datasets.register_coco_instances("val", {}, validFile, imageFolder)

NameError: ignored

In [62]:
import detectron2.data as data
train_metadata = data.MetadataCatalog.get("train")
train_data = data.DatasetCatalog.get("train")
val_data = data.DatasetCatalog.get("val")
num_trainImage = len(train_data)
num_valImage = len(val_data)

In [ ]:
print(f"{num_trainImage} training images \n"
      + f"{num_valImage} validation images {train_metadata.get('thing_classes')}" )

444 training images 
111 validation images ['band_member', 'object_e_g_backpacks_belongings', 'observer_non_band_member']


In [1]:
import detectron2.config as config
from detectron2 import model_zoo
cfg = config.get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml"))
# List of the dataset names for training. Must be registered in DatasetCatalog
# Samples from these datasets will be merged and used as one dataset.
cfg.DATASETS.TRAIN = ("train",) #can not omit "," ?
cfg.DATASETS.TEST = ("val",)

cfg.DATALOADER.NUM_WORKERS = 2

cfg.DATALOADER.ASPECT_RATIO_GROUPING = False
# Use pre-trained weights for transfer learning
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml")
cfg.SOLVER.IMS_PER_BATCH = 8

cfg.SOLVER.BASE_LR = 0.0019

max_epochs = 100
iter_per_epoch = num_trainImage // cfg.SOLVER.IMS_PER_BATCH
cfg.SOLVER.MAX_ITER = max_epochs * iter_per_epoch
cfg.SOLVER.LR_SCHEDULER_NAME = "WarmupValLossLR"
cfg.SOLVER.PATIENCE = 7

# Things 
cfg.SOLVER.AMP.ENABLED = False

# How often to save a checkpoint while training
# 0 to save best model based on minimum val loss
cfg.SOLVER.CHECKPOINT_PERIOD = 0
cfg.SOLVER.WARMUP_ITERS = 5 
# This number should be bigger than the number
# of expected objects in each frame
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = (256)
cfg.MODEL.ROI_HEADS.NUM_CLASSES = len(train_metadata.get("thing_classes"))
# NMS threshold used on RPN proposals
cfg.MODEL.RPN.NMS_THRESH = .5
# How often to check model performance on the validation set
# if iter_per_epoch then every epoch
cfg.TEST.EVAL_PERIOD = iter_per_epoch * 2
# Should be larger than max number of expected objects in image
cfg.TEST.DETECTIONS_PER_IMAGE = 200
cfg.INPUT.MIN_SIZE_TRAIN = (0)
cfg.INPUT.MIN_SIZE_TEST = (0)
cfg.INPUT.MAX_SIZE_TRAIN = (4100)
cfg.INPUT.MAX_SIZE_TEST = (4100)

# Set the data augementation regime
cfg.INPUT.CROP.ENABLED = True
cfg.INPUT.CROP.TYPE = "absolute"
cfg.INPUT.CROP.SIZE = (720.0, 720.0) # was 720, 1280
cfg.INPUT.RESIZE = False
cfg.INPUT.RESIZE_SHAPE = (720, 2048)
cfg.INPUT.VER_FLIP = True
cfg.INPUT.HOR_FLIP = True
cfg.INPUT.AUG_ON_TEST = False
cfg.INPUT.CONTRAST = True
cfg.INPUT.CONTRAST_RANGE = (.5, 1.5)
cfg.INPUT.BRIGHTNESS = True
cfg.INPUT.BRIGHTNESS_RANGE = (.3, 1.7)
cfg.INPUT.SATURATION = True
cfg.INPUT.SATURATION_RANGE = (.7, 1.4)

output_name = 'maxiter-{}_lr-{}_detectPerIm-{}_minsize-{}_batchsize-{}_nms-{}'.format(
    cfg.SOLVER.MAX_ITER, cfg.SOLVER.BASE_LR, cfg.TEST.DETECTIONS_PER_IMAGE, 
    np.min(cfg.INPUT.MIN_SIZE_TRAIN), cfg.SOLVER.IMS_PER_BATCH, cfg.MODEL.RPN.NMS_THRESH
)
cfg.OUTPUT_DIR = os.path.join(cfg.OUTPUT_DIR, output_name)
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)

# Save the config file for future reference
yaml_file = os.path.join(cfg.OUTPUT_DIR, "config.yaml")
with open(yaml_file, 'w') as file:
    file.write(cfg.dump())



NameError: ignored

In [64]:
from DetectionTrainer import DetectionTrainer
trainer = DetectionTrainer(cfg)

[03/20 18:51:26 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

In [ ]:

trainer.resume_or_load(resume=False)
trainer.train()

[03/20 18:51:39 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from https://dl.fbaipublicfiles.com/detectron2/COCO-Detection/faster_rcnn_R_50_FPN_3x/137849458/model_final_280758.pkl ...


roi_heads.box_predictor.bbox_pred.{bias, weight}
roi_heads.box_predictor.cls_score.{bias, weight}


[03/20 18:51:41 d2.engine.train_loop]: Starting training from iteration 0


/usr/local/lib/python3.9/dist-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3190.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


[03/20 18:53:00 d2.utils.events]:  eta: 2:35:35  iter: 19  total_loss: 1.406  loss_cls: 0.5463  loss_box_reg: 0.488  loss_rpn_cls: 0.1604  loss_rpn_loc: 0.2662    time: 3.4118  last_time: 1.2294  data_time: 2.2994  last_data_time: 0.0057   lr: N/A  max_mem: 6179M
[03/20 18:53:55 d2.utils.events]:  eta: 3:38:35  iter: 39  total_loss: 1.129  loss_cls: 0.3261  loss_box_reg: 0.4871  loss_rpn_cls: 0.08951  loss_rpn_loc: 0.2443    time: 3.0541  last_time: 5.1937  data_time: 1.3908  last_data_time: 3.8279   lr: N/A  max_mem: 6179M
[03/20 18:54:48 d2.utils.events]:  eta: 3:37:47  iter: 59  total_loss: 1.118  loss_cls: 0.311  loss_box_reg: 0.4674  loss_rpn_cls: 0.09913  loss_rpn_loc: 0.2503    time: 2.9143  last_time: 3.3141  data_time: 1.2421  last_data_time: 1.9803   lr: N/A  max_mem: 6179M
[03/20 18:55:42 d2.utils.events]:  eta: 3:36:59  iter: 79  total_loss: 1.034  loss_cls: 0.2618  loss_box_reg: 0.4581  loss_rpn_cls: 0.08839  loss_rpn_loc: 0.2291    time: 2.8540  last_time: 4.9341  data_ti

In [ ]:
from DetectionDatasetMapper import DetectionDatasetMapper
from detectron2.engine import DefaultPredictor
import random


In [ ]:

# Check training quality
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "best-model-config-iter-896-loss-0.8034359399295811.pth")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5
cfg.DATASETS.TEST = ("val", )
predictor = DefaultPredictor(cfg)

for d in random.sample(val_data, 3):
# for d in train_dicts:
    im = cv2.imread(d["file_name"])
    outputs = predictor(im)
    v = Visualizer(im[:, :, ::-1],
                   metadata=train_metadata,
                   scale=0.8,
                   
    )
    v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    plt.figure(figsize=(20,20))
    plt.imshow(v.get_image()[:, :, ::])

In [ ]:
from inference import register_inference_instances, nms_all_classes
from detectron2.data import build_detection_test_loader
from detectron2.modeling import build_model
from detectron2.checkpoint import DetectionCheckpointer


In [ ]:
# build model
model = build_model(cfg)
_ = model.eval()

checkpointer = DetectionCheckpointer(model)
_ = checkpointer.load(cfg.MODEL.WEIGHTS)


In [ ]:
# predict with frames extracted from the videos
pathName = '/content/drive/My Drive/Drone Footage_Coding Task/Drone Footage/11 9 flights/'
videos = glob.glob(pathName + '*.MP4')
videos = videos[0:13]
for video in videos:
  videoName = video.split('/')[7].split('.')[0]
  outFolder = '/content/drive/My Drive/Frames/'+ videoName
  resultFolder = '/content/drive/My Drive/Frames/Result'
  os.makedirs(resultFolder, exist_ok = True)
  register_inference_instances("inference", outFolder)
  mapper = DetectionDatasetMapper(cfg, is_train = False)
  loader = build_detection_test_loader(cfg, "inference", mapper = mapper)
  for batch in loader:
    image = batch[0]['image'].numpy()
    image = np.transpose(image, [1, 2, 0])
    plt.figure(figsize = (20, 20))
    plt.imshow(image[..., ::-1].astype(np.uint8))
    break
  all_detections = []

  t = time.time()
  with torch.no_grad():
      for batch_num, image_batch in enumerate(loader):
          #if batch_num % 500 == 0:
            #  print(f"{batch_num} images processed.")
          detections = model(image_batch)
          for ind, detection in enumerate(detections):
              instances = detection["instances"]
              # Postprocessing (non-maximum supression across all classes)
              instances = nms_all_classes(instances, iou_thresh=.5)
              detections[ind]["instances"] = instances

          detection_dict = detections[0]['instances'].get_fields()
          detection_dict['pred_boxes'] = detection_dict['pred_boxes'].tensor.cpu().numpy()
          detection_dict['scores'] = detection_dict['scores'].cpu().numpy()
          detection_dict['pred_classes'] = detection_dict['pred_classes'].cpu().numpy()
          detection_dict['image_name'] = os.path.basename(image_batch[0]['file_name'])
          all_detections.append(detection_dict)

  np_detections_file = os.path.join(resultFolder, '{}_detections.npy'.format(videoName))
  np.save(np_detections_file, all_detections)  
  print(time.time() - t)
  image = image_batch[0]['image'].numpy()
  image = np.transpose(image, [1, 2, 0])

  v = Visualizer(image[...,::-1],
                metadata=None,
                scale=1.0,
  )
  v = v.draw_instance_predictions(detections[0]['instances'])
  plt.figure(figsize=(20,20))
  plt.imshow(v.get_image())